###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
import sqlite3 as sq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
def percent(x, y):
    return (x/y) * 100

# Foreign born Population by Place of Birth for the GNRC Senior Centers

This data is not available on MySidewalk, where these senior center profiles are hosted. This notebook contains the API pulls and the data cleaning process necessary to include this as outside data in these profiles.

In [3]:
#ACS52021
conn = sq.connect('../../../../GNRC-Data-Pipelines/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021]', conn)
ACS52021 = pd.DataFrame(sql_query)
ACS52021.drop(columns = 'Source', inplace = True)
data = ACS52021

In [4]:
geos = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 'Humphreys County, Tennessee', 
        'Maury County, Tennessee', 'Montgomery County, Tennessee', 'Robertson County, Tennessee', 'Rutherford County, Tennessee', 'Stewart County, Tennessee', 
        'Sumner County, Tennessee', 'Trousdale County, Tennessee', 'Williamson County, Tennessee', 'Wilson County, Tennessee']

In [5]:
cols1 = ['NAME', 'Foreign Born Population' ,'ForeignBorn:Europe' ,
        'ForeignBorn:Asia' ,'ForeignBorn:Africa' ,'ForeignBorn:Oceania' ,'ForeignBorn:Americas' ,'ForeignBorn:Central & South America, Caribbean' ,
        'ForeignBorn:Canada & Other Northern American' ,'ForeignBorn:Europe:NorthernEurope' ,
        'ForeignBorn:Europe:NorthernEurope:Denmark' ,'ForeignBorn:Europe:NorthernEurope:Ireland' ,'ForeignBorn:Europe:NorthernEurope:Norway' ,
        'ForeignBorn:Europe:NorthernEurope:Sweden' ,'ForeignBorn:Europe:NorthernEurope:UK' ,'ForeignBorn:Europe:NorthernEurope:UK:Excluding England and Scotland' ,
        'ForeignBorn:Europe:NorthernEurope:UK:England' ,'ForeignBorn:Europe:NorthernEurope:UK:Scotland' ,'ForeignBorn:Europe:NorthernEurope:Other' ,
        'ForeignBorn:Europe:WesternEurope' ,'ForeignBorn:Europe:WesternEurope:Austria' ,'ForeignBorn:Europe:WesternEurope:Belgium' ,
        'ForeignBorn:Europe:WesternEurope:France' ,'ForeignBorn:Europe:WesternEurope:Germany' ,'ForeignBorn:Europe:WesternEurope:Netherlands' ,
        'ForeignBorn:Europe:WesternEurope:Switzerland' ,'ForeignBorn:Europe:WesternEurope:Other' ,'ForeignBorn:Europe:SouthernEurope' ,
        'ForeignBorn:Europe:SouthernEurope:Greece' ,'ForeignBorn:Europe:SouthernEurope:Italy' ,'ForeignBorn:Europe:SouthernEurope:Portugal' ,
        'ForeignBorn:Europe:SouthernEurope:Azores Islands' ,'ForeignBorn:Europe:SouthernEurope:Spain' ,'ForeignBorn:Europe:SouthernEurope:Other' ,
        'ForeignBorn:Europe:EasternEurope' ,'ForeignBorn:Europe:EasternEurope:Albania' ,'ForeignBorn:Europe:EasternEurope:Belarus' ,
        'ForeignBorn:Europe:EasternEurope:Boznia and Herzegovina' ,'ForeignBorn:Europe:EasternEurope:Bulgaria' ,'ForeignBorn:Europe:EasternEurope:Croatia' ,
        'ForeignBorn:Europe:EasternEurope:Czechoslovakia (Czech Republic and Slovakia)' ,'ForeignBorn:Europe:EasternEurope:Hungary' ,
        'ForeignBorn:Europe:EasternEurope:Latvia' ,'ForeignBorn:Europe:EasternEurope:Lithuania' ,'ForeignBorn:Europe:EasternEurope:Moldova' ,
        'ForeignBorn:Europe:EasternEurope:North Macedonia (Macedonia)' ,'ForeignBorn:Europe:EasternEurope:Poland' ,'ForeignBorn:Europe:EasternEurope:Romania' ,
        'ForeignBorn:Europe:EasternEurope:Russia' ,'ForeignBorn:Europe:EasternEurope:Serbia' ,'ForeignBorn:Europe:EasternEurope:Ukraine' ,
        'ForeignBorn:Europe:EasternEurope:Other' , 'ForeignBorn:Asia:EasternAsia' ,'ForeignBorn:Asia:EasternAsia:China' ,
        'ForeignBorn:Asia:EasternAsia:China Excluding Hong Kong and Taiwan' ,'ForeignBorn:Asia:EasternAsia:Hong Kong' ,'ForeignBorn:Asia:EasternAsia:Taiwan' ,
        'ForeignBorn:Asia:EasternAsia:Japan' ,'ForeignBorn:Asia:EasternAsia:Korea' ,'ForeignBorn:Asia:EasternAsia:Other' ,'ForeignBorn:Asia:SouthCentralAsia' ,
        'ForeignBorn:Asia:SouthCentralAsia:Afghanistan' ,'ForeignBorn:Asia:SouthCentralAsia:Bangladesh' ,'ForeignBorn:Asia:SouthCentralAsia:India' ,
        'ForeignBorn:Asia:SouthCentralAsia:Iran' ,'ForeignBorn:Asia:SouthCentralAsia:Kazakhstan' ,'ForeignBorn:Asia:SouthCentralAsia:Nepal' ,
        'ForeignBorn:Asia:SouthCentralAsia:Pakistan' ,'ForeignBorn:Asia:SouthCentralAsia:Sri Lanka' ,'ForeignBorn:Asia:SouthCentralAsia:Uzbekistan' ,
        'ForeignBorn:Asia:SouthCentralAsia:Other' ,'ForeignBorn:Asia:SouthEasternAsia' ,'ForeignBorn:Asia:SouthEasternAsia:Burma' ,
        'ForeignBorn:Asia:SouthEasternAsia:Cambodia' ,'ForeignBorn:Asia:SouthEasternAsia:Indonesia' ,'ForeignBorn:Asia:SouthEasternAsia:Laos' ,
        'ForeignBorn:Asia:SouthEasternAsia:Malaysia' ,'ForeignBorn:Asia:SouthEasternAsia:Philippines' ,'ForeignBorn:Asia:SouthEasternAsia:Singapore' ,
        'ForeignBorn:Asia:SouthEasternAsia:Thailand' ,'ForeignBorn:Asia:SouthEasternAsia:Vietnam' ,'ForeignBorn:Asia:SouthEasternAsia:Other' ,
        'ForeignBorn:Asia:WesternAsia' ,'ForeignBorn:Asia:WesternAsia:Armenia' ,'ForeignBorn:Asia:WesternAsia:Iraq' ,'ForeignBorn:Asia:WesternAsia:Israel' ,
        'ForeignBorn:Asia:WesternAsia:Jordan' ,'ForeignBorn:Asia:WesternAsia:Kuwait' ,'ForeignBorn:Asia:WesternAsia:Lebanon' ,
        'ForeignBorn:Asia:WesternAsia:Saudi Arabia' ,'ForeignBorn:Asia:WesternAsia:Syria' ,'ForeignBorn:Asia:WesternAsia:Turkey' ,
        'ForeignBorn:Asia:WesternAsia:Yemen' ,'ForeignBorn:Asia:WesternAsia:Other' ,'ForeignBorn:Africa:EasternAfrica' ,
        'ForeignBorn:Africa:EasternAfrica:Eritrea' ,'ForeignBorn:Africa:EasternAfrica:Ethiopia' ,'ForeignBorn:Africa:EasternAfrica:Kenya' ,
        'ForeignBorn:Africa:EasternAfrica:Somalia' ,'ForeignBorn:Africa:EasternAfrica:Uganda' ,'ForeignBorn:Africa:EasternAfrica:Zimbabwe' ,
        'ForeignBorn:Africa:EasternAfrica:Other' ,'ForeignBorn:Africa:MiddleAfrica' ,'ForeignBorn:Africa:MiddleAfrica:Cameroon' ,
        'ForeignBorn:Africa:MiddleAfrica:Congo' ,'ForeignBorn:Africa:MiddleAfrica:Democratic Republic of Congo (Zaire)' ,'ForeignBorn:Africa:MiddleAfrica:Other' ,
        'ForeignBorn:Africa:NorthernAfrica' ,'ForeignBorn:Africa:NorthernAfrica:Egypt' ,'ForeignBorn:Africa:NorthernAfrica:Morocco' ,
        'ForeignBorn:Africa:NorthernAfrica:Sudan' ,'ForeignBorn:Africa:NorthernAfrica:Other' ,'ForeignBorn:Africa:WesternAfrica' ,
        'ForeignBorn:Africa:WesternAfrica:Cabo Verde' ,'ForeignBorn:Africa:WesternAfrica:Ghana' ,'ForeignBorn:Africa:WesternAfrica:Liberia' ,
        'ForeignBorn:Africa:WesternAfrica:Nigeria' ,'ForeignBorn:Africa:WesternAfrica:Senegal' ,'ForeignBorn:Africa:WesternAfrica:Sierra Leone' ,
        'ForeignBorn:Africa:WesternAfrica:Other' ,'ForeignBorn:Oceania:Australia and New Zealand Subregion' ,
        'ForeignBorn:Oceania:Australia and New Zealand Subregion:Australia' ,'ForeignBorn:Oceania:Australia and New Zealand Subregion:Other' ,
        'ForeignBorn:Oceania:Fiji' ,'ForeignBorn:Oceania:Micronesia' ,'ForeignBorn:Central & South America, Caribbean:Caribbean' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:Bahamas' ,'ForeignBorn:Central & South America, Caribbean:Caribbean:Barbados' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:Cuba' ,'ForeignBorn:Central & South America, Caribbean:Caribbean:Dominica' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:Dominican Republic' ,'ForeignBorn:Central & South America, Caribbean:Caribbean:Grenada' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:Haiti' ,'ForeignBorn:Central & South America, Caribbean:Caribbean:Jamaica' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:St. Vincent and the Grenadines' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:Trinidad and Tobago' ,'ForeignBorn:Central & South America, Caribbean:Caribbean:West Indies' ,
        'ForeignBorn:Central & South America, Caribbean:Caribbean:Other' ,'ForeignBorn:Central & South America, Caribbean:Central America' ,
        'ForeignBorn:Central & South America, Caribbean:Central America:Belize' ,'ForeignBorn:Central & South America, Caribbean:Central America:Costa Rica' ,
        'ForeignBorn:Central & South America, Caribbean:Central America:El Salvador' ,'ForeignBorn:Central & South America, Caribbean:Central America:Guatemala' ,
        'ForeignBorn:Central & South America, Caribbean:Central America:Honduras' ,'ForeignBorn:Central & South America, Caribbean:Central America:Mexico' ,
        'ForeignBorn:Central & South America, Caribbean:Central America:Nicaragua' ,'ForeignBorn:Central & South America, Caribbean:Central America:Panama' ,
        'ForeignBorn:Central & South America, Caribbean:Central America:Other' ,'ForeignBorn:Central & South America, Caribbean:South America' ,
        'ForeignBorn:Central & South America, Caribbean:South America:Argentina' ,'ForeignBorn:Central & South America, Caribbean:South America:Bolivia' ,
        'ForeignBorn:Central & South America, Caribbean:South America:Brazil' ,'ForeignBorn:Central & South America, Caribbean:South America:Chile' ,
        'ForeignBorn:Central & South America, Caribbean:South America:Columbia' ,'ForeignBorn:Central & South America, Caribbean:South America:Ecuador' ,
        'ForeignBorn:Central & South America, Caribbean:South America:Guyana' ,'ForeignBorn:Central & South America, Caribbean:South America:Peru' ,
        'ForeignBorn:Central & South America, Caribbean:South America:Uruguay' ,'ForeignBorn:Central & South America, Caribbean:South America:Venezuela' ,
        'ForeignBorn:Central & South America, Caribbean:South America:Other' ,'ForeignBorn:Canada & Other Northern American:Canada' ,
        'ForeignBorn:Canada & Other Northern American:Other']
        
cols2 = ['NAME', 'Foreign Born Population %']
data1 = data.loc[data['NAME'].isin(geos)]
data1 = data1[cols1]
data2 = data.loc[data['NAME'].isin(geos)]
data2 = data2[cols2]

In [6]:
data1.head(2)

,NAME,Foreign Born Population,ForeignBorn:Europe,ForeignBorn:Asia,ForeignBorn:Africa,ForeignBorn:Oceania,ForeignBorn:Americas,"ForeignBorn:Central & South America, Caribbean",ForeignBorn:Canada & Other Northern American,ForeignBorn:Europe:NorthernEurope,ForeignBorn:Europe:NorthernEurope:Denmark,ForeignBorn:Europe:NorthernEurope:Ireland,ForeignBorn:Europe:NorthernEurope:Norway,ForeignBorn:Europe:NorthernEurope:Sweden,ForeignBorn:Europe:NorthernEurope:UK,ForeignBorn:Europe:NorthernEurope:UK:Excluding England and Scotland,ForeignBorn:Europe:NorthernEurope:UK:England,ForeignBorn:Europe:NorthernEurope:UK:Scotland,ForeignBorn:Europe:NorthernEurope:Other,ForeignBorn:Europe:WesternEurope,ForeignBorn:Europe:WesternEurope:Austria,ForeignBorn:Europe:WesternEurope:Belgium,ForeignBorn:Europe:WesternEurope:France,ForeignBorn:Europe:WesternEurope:Germany,ForeignBorn:Europe:WesternEurope:Netherlands,ForeignBorn:Europe:WesternEurope:Switzerland,ForeignBorn:Europe:WesternEurope:Other,ForeignBorn:Europe:SouthernEurope,ForeignBorn:Europe:SouthernEurope:Greece,ForeignBorn:Europe:SouthernEurope:Italy,ForeignBorn:Europe:SouthernEurope:Portugal,ForeignBorn:Europe:SouthernEurope:Azores Islands,ForeignBorn:Europe:SouthernEurope:Spain,ForeignBorn:Europe:SouthernEurope:Other,ForeignBorn:Europe:EasternEurope,ForeignBorn:Europe:EasternEurope:Albania,ForeignBorn:Europe:EasternEurope:Belarus,ForeignBorn:Europe:EasternEurope:Boznia and Herzegovina,ForeignBorn:Europe:EasternEurope:Bulgaria,ForeignBorn:Europe:EasternEurope:Croatia,ForeignBorn:Europe:EasternEurope:Czechoslovakia (Czech Republic and Slovakia),ForeignBorn:Europe:EasternEurope:Hungary,ForeignBorn:Europe:EasternEurope:Latvia,ForeignBorn:Europe:EasternEurope:Lithuania,ForeignBorn:Europe:EasternEurope:Moldova,ForeignBorn:Europe:EasternEurope:North Macedonia (Macedonia),ForeignBorn:Europe:EasternEurope:Poland,ForeignBorn:Europe:EasternEurope:Romania,ForeignBorn:Europe:EasternEurope:Russia,ForeignBorn:Europe:EasternEurope:Serbia,ForeignBorn:Europe:EasternEurope:Ukraine,ForeignBorn:Europe:EasternEurope:Other,ForeignBorn:Asia:EasternAsia,ForeignBorn:Asia:EasternAsia:China,ForeignBorn:Asia:EasternAsia:China Excluding Hong Kong and Taiwan,ForeignBorn:Asia:EasternAsia:Hong Kong,ForeignBorn:Asia:EasternAsia:Taiwan,ForeignBorn:Asia:EasternAsia:Japan,ForeignBorn:Asia:EasternAsia:Korea,ForeignBorn:Asia:EasternAsia:Other,ForeignBorn:Asia:SouthCentralAsia,ForeignBorn:Asia:SouthCentralAsia:Afghanistan,ForeignBorn:Asia:SouthCentralAsia:Bangladesh,ForeignBorn:Asia:SouthCentralAsia:India,ForeignBorn:Asia:SouthCentralAsia:Iran,ForeignBorn:Asia:SouthCentralAsia:Kazakhstan,ForeignBorn:Asia:SouthCentralAsia:Nepal,ForeignBorn:Asia:SouthCentralAsia:Pakistan,ForeignBorn:Asia:SouthCentralAsia:Sri Lanka,ForeignBorn:Asia:SouthCentralAsia:Uzbekistan,ForeignBorn:Asia:SouthCentralAsia:Other,ForeignBorn:Asia:SouthEasternAsia,ForeignBorn:Asia:SouthEasternAsia:Burma,ForeignBorn:Asia:SouthEasternAsia:Cambodia,ForeignBorn:Asia:SouthEasternAsia:Indonesia,ForeignBorn:Asia:SouthEasternAsia:Laos,ForeignBorn:Asia:SouthEasternAsia:Malaysia,ForeignBorn:Asia:SouthEasternAsia:Philippines,ForeignBorn:Asia:SouthEasternAsia:Singapore,ForeignBorn:Asia:SouthEasternAsia:Thailand,ForeignBorn:Asia:SouthEasternAsia:Vietnam,ForeignBorn:Asia:SouthEasternAsia:Other,ForeignBorn:Asia:WesternAsia,ForeignBorn:Asia:WesternAsia:Armenia,ForeignBorn:Asia:WesternAsia:Iraq,ForeignBorn:Asia:WesternAsia:Israel,ForeignBorn:Asia:WesternAsia:Jordan,ForeignBorn:Asia:WesternAsia:Kuwait,ForeignBorn:Asia:WesternAsia:Lebanon,ForeignBorn:Asia:WesternAsia:Saudi Arabia,ForeignBorn:Asia:WesternAsia:Syria,ForeignBorn:Asia:WesternAsia:Turkey,ForeignBorn:Asia:WesternAsia:Yemen,ForeignBorn:Asia:WesternAsia:Other,ForeignBorn:Africa:EasternAfrica,ForeignBorn:Africa:EasternAfrica:Eritrea,ForeignBorn:Africa:EasternAfrica:Ethiopia,ForeignBorn:Africa:EasternAfrica:Kenya,ForeignBorn:Africa:EasternAfrica:Somalia,ForeignBorn:Africa:EasternAfrica:Uganda,ForeignBorn:Africa:EasternAfrica:Zimbabw

In [7]:
data2

,NAME,Foreign Born Population %
0,"Cheatham County, Tennessee",3.111698
1,"Davidson County, Tennessee",13.417832
2,"Dickson County, Tennessee",1.618519
3,"Houston County, Tennessee",0.910415
4,"Humphreys County, Tennessee",1.667018
6,"Maury County, Tennessee",3.761198
7,"Montgomery County, Tennessee",5.919361
8,"Robertson County, Tennessee",4.652256
9,"Rutherford County, Tennessee",8.050180
10,"Stewart County, Tennessee",2.365686


In [8]:
cols = data1.columns
data1 = data1.melt(id_vars = 'NAME', value_vars = cols, var_name = 'Population Group', value_name = 'Population').sort_values(by = 'NAME')
cols = data2.columns
data2 = data2.melt(id_vars = 'NAME', value_vars = cols, var_name = 'Population Group', value_name = 'Percent of Population').sort_values(by = 'NAME')
data1 = data1.sort_values(by = 'Population Group')
data2 = data2.sort_values(by = 'Population Group')

In [9]:
data1.head()

,NAME,Population Group,Population
0,"Cheatham County, Tennessee",Foreign Born Population,1272.0
2,"Dickson County, Tennessee",Foreign Born Population,874.0
4,"Humphreys County, Tennessee",Foreign Born Population,316.0
5,"Maury County, Tennessee",Foreign Born Population,3724.0
7,"Robertson County, Tennessee",Foreign Born Population,3364.0


In [25]:
df = data1.loc[data1['NAME'] == 'Wilson County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 7395)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Wilson County, Tennessee",Foreign Born Population,7395.0,100.000000
28,"Wilson County, Tennessee",ForeignBorn:Americas,3365.0,45.503719
75,"Wilson County, Tennessee","ForeignBorn:Central & South America, Caribbean",3148.0,42.569304
89,"Wilson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",2187.0,29.574037
29,"Wilson County, Tennessee",ForeignBorn:Asia,1974.0,26.693712
95,"Wilson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",1612.0,21.798513
1,"Wilson County, Tennessee",ForeignBorn:Africa,1071.0,14.482759
38,"Wilson County, Tennessee",ForeignBorn:Asia:SouthCentralAsia,1028.0,13.901285
111,"Wilson County, Tennessee",ForeignBorn:Europe,985.0,13.319811
41,"Wilson County, Tennessee",ForeignBorn:Asia:SouthCentralAsia:India,909.0,12.292089


In [26]:
df.to_csv('Wilson.csv', index = False)

In [27]:
df = data1.loc[data1['NAME'] == 'Houston County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 75)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Houston County, Tennessee",Foreign Born Population,75.0,100.000000
29,"Houston County, Tennessee",ForeignBorn:Asia,30.0,40.000000
30,"Houston County, Tennessee",ForeignBorn:Asia:EasternAsia,27.0,36.000000
31,"Houston County, Tennessee",ForeignBorn:Asia:EasternAsia:China,27.0,36.000000
32,"Houston County, Tennessee",ForeignBorn:Asia:EasternAsia:China Excluding H...,27.0,36.000000
147,"Houston County, Tennessee",ForeignBorn:Europe:WesternEurope,23.0,30.666667
111,"Houston County, Tennessee",ForeignBorn:Europe,23.0,30.666667
28,"Houston County, Tennessee",ForeignBorn:Americas,22.0,29.333333
73,"Houston County, Tennessee",ForeignBorn:Canada & Other Northern American:C...,18.0,24.000000
72,"Houston County, Tennessee",ForeignBorn:Canada & Other Northern American,18.0,24.000000


In [10]:
df.to_csv('cheatham.csv')

In [11]:
df = data1.loc[data1['NAME'] == 'Davidson County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 95064)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Davidson County, Tennessee",Foreign Born Population,95064.0,100.000000
28,"Davidson County, Tennessee",ForeignBorn:Americas,39950.0,42.024320
75,"Davidson County, Tennessee","ForeignBorn:Central & South America, Caribbean",37950.0,39.920475
89,"Davidson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",33688.0,35.437179
29,"Davidson County, Tennessee",ForeignBorn:Asia,27745.0,29.185601
1,"Davidson County, Tennessee",ForeignBorn:Africa,20411.0,21.470799
95,"Davidson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",19674.0,20.695531
15,"Davidson County, Tennessee",ForeignBorn:Africa:NorthernAfrica,8071.0,8.490070
60,"Davidson County, Tennessee",ForeignBorn:Asia:WesternAsia,7941.0,8.353320
49,"Davidson County, Tennessee",ForeignBorn:Asia:SouthEasternAsia,7632.0,8.028276


In [13]:
df.to_csv('Davidson.csv', index = False)

In [142]:
df = data1.loc[data1['NAME'] == 'Dickson County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 874)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Dickson County, Tennessee",Foreign Born Population,874.0,100.000000
28,"Dickson County, Tennessee",ForeignBorn:Americas,428.0,48.970252
75,"Dickson County, Tennessee","ForeignBorn:Central & South America, Caribbean",370.0,42.334096
89,"Dickson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",292.0,33.409611
95,"Dickson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",237.0,27.116705
111,"Dickson County, Tennessee",ForeignBorn:Europe,194.0,22.196796
29,"Dickson County, Tennessee",ForeignBorn:Asia,185.0,21.167048
49,"Dickson County, Tennessee",ForeignBorn:Asia:SouthEasternAsia,184.0,21.052632
130,"Dickson County, Tennessee",ForeignBorn:Europe:NorthernEurope,146.0,16.704805
137,"Dickson County, Tennessee",ForeignBorn:Europe:NorthernEurope:UK:England,134.0,15.331808


In [144]:
df = data1.loc[data1['NAME'] == 'Maury County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 3724)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Maury County, Tennessee",Foreign Born Population,3724.0,100.000000
28,"Maury County, Tennessee",ForeignBorn:Americas,2662.0,71.482277
75,"Maury County, Tennessee","ForeignBorn:Central & South America, Caribbean",2462.0,66.111708
89,"Maury County, Tennessee","ForeignBorn:Central & South America, Caribbean...",1984.0,53.276047
95,"Maury County, Tennessee","ForeignBorn:Central & South America, Caribbean...",1861.0,49.973147
29,"Maury County, Tennessee",ForeignBorn:Asia,681.0,18.286788
111,"Maury County, Tennessee",ForeignBorn:Europe,357.0,9.586466
99,"Maury County, Tennessee","ForeignBorn:Central & South America, Caribbean...",341.0,9.156821
38,"Maury County, Tennessee",ForeignBorn:Asia:SouthCentralAsia,273.0,7.330827
110,"Maury County, Tennessee","ForeignBorn:Central & South America, Caribbean...",270.0,7.250269


In [146]:
df = data1.loc[data1['NAME'] == 'Montgomery County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 12796)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Montgomery County, Tennessee",Foreign Born Population,12796.0,100.000000
28,"Montgomery County, Tennessee",ForeignBorn:Americas,5697.0,44.521726
75,"Montgomery County, Tennessee","ForeignBorn:Central & South America, Caribbean",5226.0,40.840888
29,"Montgomery County, Tennessee",ForeignBorn:Asia,3735.0,29.188809
89,"Montgomery County, Tennessee","ForeignBorn:Central & South America, Caribbean...",3526.0,27.555486
95,"Montgomery County, Tennessee","ForeignBorn:Central & South America, Caribbean...",2248.0,17.567990
111,"Montgomery County, Tennessee",ForeignBorn:Europe,1728.0,13.504220
1,"Montgomery County, Tennessee",ForeignBorn:Africa,1510.0,11.800563
30,"Montgomery County, Tennessee",ForeignBorn:Asia:EasternAsia,1492.0,11.659894
49,"Montgomery County, Tennessee",ForeignBorn:Asia:SouthEasternAsia,1184.0,9.252892


In [147]:
df.to_csv('montgomery.csv')

In [153]:
df = data1.loc[data1['NAME'] == 'Robertson County, Tennessee'].reset_index(drop = True)
df['Percent of Foreign Born Population'] = percent(df['Population'], 3364)
df = df.sort_values(by = 'Percent of Foreign Born Population', ascending = False)
df.head(20)

,NAME,Population Group,Population,Percent of Foreign Born Population
0,"Robertson County, Tennessee",Foreign Born Population,3364.0,100.000000
28,"Robertson County, Tennessee",ForeignBorn:Americas,2577.0,76.605232
75,"Robertson County, Tennessee","ForeignBorn:Central & South America, Caribbean",2391.0,71.076100
89,"Robertson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",2038.0,60.582640
95,"Robertson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",1474.0,43.816885
29,"Robertson County, Tennessee",ForeignBorn:Asia,419.0,12.455410
93,"Robertson County, Tennessee","ForeignBorn:Central & South America, Caribbean...",279.0,8.293698
111,"Robertson County, Tennessee",ForeignBorn:Europe,251.0,7.461356
30,"Robertson County, Tennessee",ForeignBorn:Asia:EasternAsia,199.0,5.915577
73,"Robertson County, Tennessee",ForeignBorn:Canada & Other Northern American:C...,186.0,5.529132


In [154]:
df.to_csv('Robertson.csv', index = False)